In [ ]:
import numpy as np
import os
import tarfile
import tensorflow as tf
import cv2

In [ ]:
# Function provided by CIFAR dataset creators to unpackage files
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def merge_two_dicts(x, y):
    z = x.copy()   # start with keys and values of x
    z.update(y)    # modifies z with keys and values of y
    return z

## Data Loading and Prep

In [3]:
train = unpickle('data\cifar-10-batches-py\data_batch_1')
for i in range(4):
     batch = unpickle(f'data\cifar-10-batches-py\data_batch_{i+2}')
     train = merge_two_dicts(train, batch)
train[b'labels'] = np.asarray(train[b'labels'])

test = unpickle('data\cifar-10-batches-py\\test_batch')
test[b'labels'] = np.asarray(test[b'labels'])

classes = unpickle('data\cifar-10-batches-py\\batches.meta')[b'label_names']

In [4]:
'''
Change data from (10000, 3072) array to (10000, 32, 32, 1) greyscale images
Greyscale is better since the color of the images won't assist in classification.
Greyscale is also faster when training and predicting.
'''

new_data = np.zeros((10000, 32, 32, 3))
data = train[b'data']

r = np.reshape(data[:, 0:1024], (10000, 32, 32))
g = np.reshape(data[:, 1024:2048], (10000, 32, 32))
b = np.reshape(data[:, 2048:3072], (10000, 32, 32))
new_data[:, :, :, 2] = r
new_data[:, :, :, 1] = g
new_data[:, :, :, 0] = b

grey = np.zeros((10000, 32, 32, 1))
for i in range(len(new_data)):
    grey[i, :, :, 0] = cv2.cvtColor(new_data[i].astype(np.uint8), cv2.COLOR_BGR2GRAY)
    
# Add reshaped data back to dictionary while dividing by 255 to normalize
train[b'data'] = grey/255.0

In [5]:
# Repeat for test dataset
size = len(test[b'labels'])
new_data = np.zeros((size, 32, 32, 3))
data = test[b'data']

r = np.reshape(data[:, 0:1024], (size, 32, 32))
g = np.reshape(data[:, 1024:2048], (size, 32, 32))
b = np.reshape(data[:, 2048:3072], (size, 32, 32))
new_data[:, :, :, 2] = r
new_data[:, :, :, 1] = g
new_data[:, :, :, 0] = b

grey = np.zeros((size, 32, 32, 1))
for i in range(len(new_data)):
    grey[i, :, :, 0] = cv2.cvtColor(new_data[i].astype(np.uint8), cv2.COLOR_BGR2GRAY)


test[b'data'] = grey/255.0

## Model Definition

In [19]:
num_classes = len(classes)
input_shape = (10000, 32, 32, 1)
'''
Defining the model below. Uses 4 layers of 2d CNN's with a 
convolution window of height and width of 3 and 32 filters.
Values are good for providing max accuracy while avoiding overfitting 
and limiting training and prediction time.
Padding is set to "same" to preserve information at the edges since
the dataset includes images with the subject is not centered.
'''
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=input_shape[1:]),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [20]:
# Adam optimizer used since it is the best for image classification
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

## Training

In [21]:
# Minibatch gradient descent used with 64 batch size and a 20% holdout size
# Epochs set to 25 since that is where validation accuracy peaks
batch_size = 64
model.fit(train[b'data'], train[b'labels'],
          batch_size=batch_size,
          validation_split=0.2,
          epochs=25,
          verbose=2
          )

Epoch 1/25
141/141 - 6s - loss: 2.1335 - accuracy: 0.1944 - val_loss: 1.8768 - val_accuracy: 0.3150 - 6s/epoch - 44ms/step
Epoch 2/25
141/141 - 6s - loss: 1.8082 - accuracy: 0.3228 - val_loss: 1.6952 - val_accuracy: 0.3810 - 6s/epoch - 41ms/step
Epoch 3/25
141/141 - 5s - loss: 1.7046 - accuracy: 0.3646 - val_loss: 1.6179 - val_accuracy: 0.3900 - 5s/epoch - 38ms/step
Epoch 4/25
141/141 - 5s - loss: 1.6123 - accuracy: 0.4050 - val_loss: 1.7150 - val_accuracy: 0.3560 - 5s/epoch - 35ms/step
Epoch 5/25
141/141 - 5s - loss: 1.5236 - accuracy: 0.4461 - val_loss: 1.5503 - val_accuracy: 0.4460 - 5s/epoch - 38ms/step
Epoch 6/25
141/141 - 6s - loss: 1.4514 - accuracy: 0.4672 - val_loss: 1.4514 - val_accuracy: 0.4690 - 6s/epoch - 39ms/step
Epoch 7/25
141/141 - 5s - loss: 1.3699 - accuracy: 0.5043 - val_loss: 1.3886 - val_accuracy: 0.5170 - 5s/epoch - 37ms/step
Epoch 8/25
141/141 - 5s - loss: 1.3157 - accuracy: 0.5308 - val_loss: 1.3542 - val_accuracy: 0.5260 - 5s/epoch - 38ms/step
Epoch 9/25
141/1

In [22]:
# Accuracy should come out to mid to high 50's
test_loss, test_acc = model.evaluate(test[b'data'], test[b'labels'], verbose=2)

print('\nTest accuracy:', test_acc)

313/313 - 2s - loss: 1.3951 - accuracy: 0.5708 - 2s/epoch - 7ms/step

Test accuracy: 0.5708000063896179


In [46]:
# Softmax used since there are more than 2 classes
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [ ]:
# Prediction accuracy increases using images with clearer subjects

#Enter .jpg image from prediction-images folder to get a prediction using model
image_name = input('Enter image name: ')

image = cv2.imread('prediction-images\\'+image_name+'.jpg')
image = cv2.resize(image, (32, 32))
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = np.array([image])
image = image/255.0
predictions = probability_model.predict(image)

print(classes[np.argmax(predictions[0])].decode("utf-8"))